In [1]:
import os
import pandas as pd
import requests
import bs4
import lxml
import functions.web_scrapper_functions as scrpr
import mysql.connector
import json
import datetime

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import logging
import warnings
warnings.filterwarnings("ignore")

logging.basicConfig(filename='Parfum AE Web Scrapper Log.log', level=logging.INFO,
format='%(asctime)s:%(levelname)s:%(message)s')

In [2]:
frag_type_dict = scrpr.get_frag_types()
print("Available fragrance types: \n", frag_type_dict.keys())

Available fragrance types: 
 dict_keys(['Body Mist', 'Car Perfume Diffuser', 'Perfume Oils', 'Our Creations', 'Dahn Al Oud', 'Interior Perfume', 'Musk Oil', 'Perfume Reed Diffuser'])


In [3]:
prod_list_dict = {}
start_time = datetime.datetime.now()

for key, val in frag_type_dict.items():
    try:
        logging.info('Currently scrapping ', key)
        prod_list_dict[key] = scrpr.prod_list(val)
        logging.info('Number of products: ', len(prod_list_dict[key]))
    except:
        logging.error("Error in ", key)

end_time = datetime.datetime.now()
time_taken = end_time - start_time
logging.info(f"Time taken to retrieve list of all products: {time_taken}")

--- Logging error ---
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\logging\__init__.py", line 1081, in emit
    msg = self.format(record)
  File "C:\ProgramData\Anaconda3\lib\logging\__init__.py", line 925, in format
    return fmt.format(record)
  File "C:\ProgramData\Anaconda3\lib\logging\__init__.py", line 664, in format
    record.message = record.getMessage()
  File "C:\ProgramData\Anaconda3\lib\logging\__init__.py", line 369, in getMessage
    msg = msg % self.args
TypeError: not all arguments converted during string formatting
Call stack:
  File "C:\ProgramData\Anaconda3\lib\runpy.py", line 194, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "C:\ProgramData\Anaconda3\lib\runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "c:\users\shahe\onedrive\documents\projects\project 4 - parfum.ae analysis\parfumaeproject\lib\site-packages\ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  Fi

In [9]:
start_time = datetime.datetime.now()
products = scrpr.prod_info(prod_list_dict)
end_time = datetime.datetime.now()
time_taken = end_time - start_time
logging.info(f"Time taken to retrieve information for each product: {time_taken}")

Our Creation of Paco Rabanne's One Million Body Mist
Our Creation of Creed's Aventus Body Mist 150ml
Our Creation of Ajmal's Amber Wood Body Mist 150ml
Our Creation of Creed's Viking Body Mist 150ml
Our Creation of LV's Imagination Body Mist
Our Creation of Bvlgari's Falkar Body Mist 150ml
Our Creation of Zilinski & Rozan's Vetiver & Lemon, Bergamot Body Mist 150ml
Our Creation of Dior's Ambre Nuit Body Mist 150ml
Our Creation of Kilian's Good Girl Gone Bad Body Mist 150ml
Our Creation of TF's Sahara Noir Body Mist
Our Creation of TF's Noir Body Mist
Our Creation of Dior's J'adore Body Mist 150ml
Our Creation of YSL's Manifesto Body Mist 150ml
Our Creation of Memo Paris' Irish Leather Body Mist 150ml
Our Creation of Creed's Aventus Cologne for Men Body Mist 150ml
Our Creation of Giorgio Armani's Si Passione Body Mist 150ml
Our Creation of Nishane's Ani Body Mist 150ml
Our Creation of LV's Afternoon Body Mist 150ml
Our Creation of Chanel's Allure Sensuelle Body Mist 150ml
Our Creation o

In [129]:
def product_reviews(prod_id):
    # Make request for the initial review page
    url_main = f'https://www.parfum.ae/index.php?route=product/product/review&product_id={prod_id}&page=1'
    r_main = requests.get(url_main)
    s_main = bs4.BeautifulSoup(r_main.content, 'lxml')
    
    # Extract review data
    reviews = []
    page = 1
    
    while True:
        url = f'https://www.parfum.ae/index.php?route=product/product/review&product_id={prod_id}&page={page}'
        r = requests.get(url)
        s = bs4.BeautifulSoup(r.content, 'lxml')
    
        for table in s.select("table"):
            reviewer_name = table.select("strong")[0].text.replace('\xa0',' ')
            reviewer_text = table.select("p")[0].text)
            reviewer_rating = len(table.select(".fa.fa-star.fa-stack-2x")))
            review.append([reviewer_name, reviewer_text, reviewer_rating])
    
        if len(s.select('table')) == 0:
            break
    
        page += 1

    return reviews

Nada Z
Extremely happy with the overall quality and delivery. Would definitely be purchasing again 
5
Yousef
The best part is that the quality of the perfume is exceptionally well. You cannot tell the difference between the original and this perfume in a customized bottle.  Very much happy saving money and getting the best quality. 10/10
5
Ali Moeen
Not very good not very bad. Satisfactory
4
Nawazish
Gave it a first try buying perfume from here. Had a great experience. The quality of the perfume is definitely more than 80% matching the original one. So yes you can surely save money here by buying good stuff.
4
Maryam
A little bit off as i didnt like the fragrance much but yes the quality of perfume is not really bad. So surely going to try other perfume purchase next time.
3
Oumar
Extremely happy with the notes and overall quality. And above all i have previously tried the clone perfumes from another brand. But one thing is for sure the quality perfum.ae is providing is way too good. H

In [64]:
# Make request for the review pages
url = f'https://www.parfum.ae/index.php?route=product/product/review&product_id=1692&page=1'
r = requests.get(url)
s = bs4.BeautifulSoup(r.contents, 'lxml')


# Extract review data
reviews = []
page=1

while len(s.select('table')) > 0:
    url_2 = f'https://www.parfum.ae/index.php?route=product/product/review&product_id={prod_id}&page={page}'
    r2 = requests.get(url_2)
    s2 = bs4.BeautifulSoup(r2.contents, 'lxml')
    
    for table in s2.select("table"):
        reviewer_name = table.select("strong")[0].text.replace('\xa0',' ')
        reviewer_text = table.select("p")[0].text
        reviewer_rating = len(table.select(".fa.fa-star.fa-stack-2x"))
        reviews.append([reviewer_name, reviewer_text, reviewer_rating])

    page+=1


In [65]:
reviews

[['muhsen ali',
  'اشتريته ضمن عينات للتجربة ؛ حقاً انها مضيعة للمال ؛ رائحة سيئة جداً ومتعجرفة ',
  1],
 ['Maria Nusheva',
  'Smells so bad, doesn’t last even 5mins. Such a waste of money! First and last order, definitely!',
  1],
 ['Asad', 'This is very amazing and can be use for any season', 5],
 ['sajal',
  'The fragrance is perfect for those who want to feel sophisticated and elegant.',
  5],
 ['ahad',
  'The fragrance is perfect for those who want to make a statement with their scent.',
  5]]

In [68]:
soup.select("div#review")[0].select(".pagination")[0]

<ul class="pagination"><li class="active"><span>1</span></li><li><a href="https://www.parfum.ae/index.php?route=product/product/review&amp;product_id=430&amp;page=2">2</a></li><li><a href="https://www.parfum.ae/index.php?route=product/product/review&amp;product_id=430&amp;page=3">3</a></li><li><a href="https://www.parfum.ae/index.php?route=product/product/review&amp;product_id=430&amp;page=4">4</a></li><li><a href="https://www.parfum.ae/index.php?route=product/product/review&amp;product_id=430&amp;page=5">5</a></li><li><a href="https://www.parfum.ae/index.php?route=product/product/review&amp;product_id=430&amp;page=6">6</a></li><li><a href="https://www.parfum.ae/index.php?route=product/product/review&amp;product_id=430&amp;page=7">7</a></li><li><a href="https://www.parfum.ae/index.php?route=product/product/review&amp;product_id=430&amp;page=8">8</a></li><li><a href="https://www.parfum.ae/index.php?route=product/product/review&amp;product_id=430&amp;page=9">9</a></li><li><a class="next"

In [118]:
r = requests.get("https://www.parfum.ae/index.php?route=product/product/review&product_id=1692&page=3")
s = bs4.BeautifulSoup(r.content, 'lxml')

In [122]:
s.select('table')

[]

In [119]:
for table in s.select("table"):
    print(table.select("strong")[0].text.replace('\xa0', ' '))
    print(table.select("p")[0].text)
    print(len(table.select(".fa.fa-star.fa-stack-2x")))


In [95]:
s.select('[data-product_id]')[0].get('data-product_id')

'1692'

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup

# Initialize the web driver (make sure you have the appropriate driver installed)
driver = webdriver.Chrome()
url = "https://www.parfum.ae/dior-ambre-nuit/#"
driver.get(url)

# Click the "Review" tab
tab_selector = driver.find_element(By.ID, "tab-review-tab")
tab_selector.click()

# Wait for the dynamic content to load (adjust the timeout as needed)
wait = WebDriverWait(driver, 10)
element = wait.until(EC.presence_of_element_located((By.ID, "tab-review")))

# Extract review data
reviews = []

# Define the total number of pages to scrape
total_pages = 9  # Update with the total number of pages

for page_number in range(1, total_pages + 1):
    # Navigate to each page
    if page_number != 1:
        page_url = f"https://www.parfum.ae/index.php?route=product/product/review&product_id={prod_id}&page={page_number}"
        driver.get(page_url)

    # Extract data from the selected tab
    page_content = driver.page_source
    soup = BeautifulSoup(page_content, "lxml")

    # Process the reviews from the current page
    for table in soup.select("table"):
        reviewer_name = table.select("strong")[0].text.replace('\xa0', ' ')
        reviewer_text = table.select("p")[0].text
        reviewer_rating = len(table.select(".fa.fa-star.fa-stack-2x"))
        reviews.append([reviewer_name, reviewer_text, reviewer_rating])

# Close the web driver when you're done
driver.quit()

# Process the extracted reviews as needed


In [34]:
reviews

{'Sabrina ': 'Actually I like it this product good job team Parfum.ae',
 'Abden': 'Good Product',
 'noor  Almazrouei': 'Really its very good and good for summer! ',
 'liyaan': "This is exactly what I was looking for. Crisp, clean, woody, bitter and medicinal with the slightly cool mintiness. This has set the bar for me personally and is the scent of kyara I\\'ve heard described and knew I had to try. development on the skin is divine. Probably best not to use before bed, it\\'s strong! No idea how the price is so reasonable, baffles me. I know I\\'ll be searching for a long time for something cultivated and dirt cheap that smells like this so I can slather it on daily. ",
 'Rufat Abdullayev': 'A fabulous Oud oil, thick like honey, among the best agarwoody perfumes '}

In [36]:
product_reviews('https://www.parfum.ae/oud-cambodi-synthetic/')

[{'Sabrina ': 'Actually I like it this product good job team Parfum.ae'},
 {'Abden': 'Good Product'},
 {'noor  Almazrouei': 'Really its very good and good for summer! '},
 {'liyaan': "This is exactly what I was looking for. Crisp, clean, woody, bitter and medicinal with the slightly cool mintiness. This has set the bar for me personally and is the scent of kyara I\\'ve heard described and knew I had to try. development on the skin is divine. Probably best not to use before bed, it\\'s strong! No idea how the price is so reasonable, baffles me. I know I\\'ll be searching for a long time for something cultivated and dirt cheap that smells like this so I can slather it on daily. "},
 {'Rufat Abdullayev': 'A fabulous Oud oil, thick like honey, among the best agarwoody perfumes '}]

In [23]:
for table in soup.select("div#review")[0].select("table"):
    print("Name: ", table.select("strong")[0].text)
    print("Review: ", table.select("p")[0].text)

Name:  Sabrina 
Review:  Actually I like it this product good job team Parfum.ae
Name:  Abden
Review:  Good Product
Name:  noor  Almazrouei
Review:  Really its very good and good for summer! 
Name:  liyaan
Review:  This is exactly what I was looking for. Crisp, clean, woody, bitter and medicinal with the slightly cool mintiness. This has set the bar for me personally and is the scent of kyara I\'ve heard described and knew I had to try. development on the skin is divine. Probably best not to use before bed, it\'s strong! No idea how the price is so reasonable, baffles me. I know I\'ll be searching for a long time for something cultivated and dirt cheap that smells like this so I can slather it on daily. 
Name:  Rufat Abdullayev
Review:  A fabulous Oud oil, thick like honey, among the best agarwoody perfumes 


In [32]:
soup.select("div#review")[0].select("table")[0].select("p")[0].text

'Actually I like it this product good job team Parfum.ae'

In [ ]:
conn = mysql.connector.connect(
    host = "localhost",
    user = os.environ.get("MYSQL_USERNAME"),
    password = os.environ.get("MYSQL_PASSWORD"),
    database = "parfumae_products"
)

cursor = conn.cursor()

In [ ]:

try:
    for key, val in frag_type_dict.items():
        serialized_value = json.dumps(val)

        insert_query = "INSERT INTO fragrance_types (name, url) VALUES (%s, %s)"
        cursor.execute(insert_query, (key, serialized_value))
    
    conn.commit()

except mysql.connector.Error as err:
    print(f"Error: {err}")
    conn.rollback()

cursor.close()
conn.close()